# Enter an Activity Id and See a Summary of that Activity

In [ ]:
from IPython.display import display, Markdown
import snakemd

import fitfile
from garmindb import ConfigManager
from garmindb.garmindb import GarminDb, Attributes, ActivitiesDb, Activities, ActivityLaps, ActivityRecords

activity_id = input('Enter the id of the activity you would like to display')

doc = snakemd.new_doc("activity")

db_params_dict = ConfigManager.get_db_params()
garmin_db = GarminDb(db_params_dict)
garmin_act_db = ActivitiesDb(db_params_dict)
measurement_system = Attributes.measurements_type(garmin_db)
unit_strings = fitfile.units.unit_strings[measurement_system]
distance_units = unit_strings[fitfile.units.UnitTypes.distance_long]
altitude_units = unit_strings[fitfile.units.UnitTypes.altitude]
temp_units = unit_strings[fitfile.units.UnitTypes.tempurature]

activity = Activities.get(garmin_act_db, activity_id)

doc.add_header(activity.name)
doc.add_paragraph(f'Id: {activity_id} Sport: {activity.sport} Activity type: {activity.type}')
if activity.description:
    doc.add_paragraph(activity.description)

doc.add_header("Summary", 3)
doc.add_table([f'Distance ({distance_units})', "Ellapsed Time", "Moving Time", "Calories", f"Average Tempature (degrees {temp_units})"],
              [[activity.distance, activity.elapsed_time, activity.moving_time, activity.calories, activity.avg_temperature]])

doc.add_header("Laps", 3)
laps = ActivityLaps.get_activity(garmin_act_db, activity_id)
lap_stats = []
for lap in laps:
    lap_stats.append([lap.lap, lap.distance, lap.elapsed_time, lap.moving_time, lap.calories, lap.avg_temperature])
doc.add_table(['Lap', f'Distance ({distance_units})', "Ellapsed Time", "Moving Time", "Calories", f"Average Tempature (degrees {temp_units})"], lap_stats)

doc.add_header("Altitude", 3)
doc.add_table([f'Gained ({altitude_units})', f'Lost ({altitude_units})'], [[activity.ascent, activity.descent]])

doc.add_header("Heart Rate", 3)
doc.add_table(['Average', 'Maximum'], [[activity.avg_hr, activity.max_hr]])

doc.add_header("Respiration Rate", 3)
doc.add_table(['Average', 'Maximum'], [[activity.avg_rr, activity.max_rr]])

doc.add_header("Time In Heart Rate Zones", 3)
doc.add_table(
    ['Zone', 'HR Threshold', "Time"],
    [
        ["1", activity.hrz_1_hr, activity.hrz_1_time],
        ["2", activity.hrz_2_hr, activity.hrz_2_time],
        ["3", activity.hrz_3_hr, activity.hrz_3_time],
        ["4", activity.hrz_4_hr, activity.hrz_4_time],
        ["5", activity.hrz_5_hr, activity.hrz_5_time],
    ])

doc.add_header("Training Effect", 3)
doc.add_table(
    ['Type', 'Rating'],
    [
        ["Aerobic", activity.training_effect],
        ["Anaerobic", activity.anaerobic_training_effect],
    ])

display(Markdown(str(doc)))